# Step 2: Extracting occupational data from inscriptions

*AIM*: This script extract the occupational data from inscriptions.

References:
1) `Harris EM. Workshop, Marketplace and Household: The Nature of Technical Specialization in Classical Athens and its Influence on Economy and Society. In: Carledge P, Cohen EE, Foxhall L, editors. Money, Labour and Land: Approaches to the Economy of Ancient Greece. London—New York: Routledge; 2001. pp. 67–99.`
2) `van Leeuwen MHD, Maas I, Miles A. HISCO: Historical International Standard Classification of Occupations. 2022 2002 [cited 27 Jan 2022]. Available: https://historyofwork.iisg.nl/`


This script was originally published by `Kaše V, Heřmánková P, Sobotková A (2022) Division of labor, specialization and diversity in the ancient Roman cities: A quantitative approach to Latin epigraphy. PLoS ONE 17(6): e0269869. https://doi.org/10.1371/journal.pone.0269869` under a CC BY-SA 4.0 International License. 

https://github.com/sdam-au/social_diversity

The *Past Social Networks Project* adapted the script to fit the needs of the project research agenda.

## Data:

**IN**:

1) Declined occupations `occups_declined_dict.json`

2) Occupations with metadata `occupations_list_hisco.csv`

3) Inscriptions `LIST_v1-0.parquet` or `https://zenodo.org/records/8431323`


**OUT**: 

1) Counts of occupations `occupations_counts.csv`

3) Inscriptions with occupations and their categorisations by HISCO and Harris `LIST_occups.parquet`

# Requirements

In [1]:
import json
import numpy as np
import re
import pandas as pd
import geopandas as gpd
import nltk
pd.options.display.max_columns = 1000 # to see all columns
import warnings
warnings.filterwarnings('ignore')
import sddk
#import matplotlib.pyplot as plt
#import matplotlib.colors as mcolors
#import geoplot as gplt

# Loading datasets

In [2]:
# local version if you have it already locally
LIST = gpd.read_parquet("../../data/large_data/LIST_v1-0.parquet") # download manually from from https://zenodo.org/records/8431323 and save to data/large_data folder

In [23]:
#LIST = gpd.read_parquet("https://zenodo.org/records/8431323/files/LIST_v1-0.parquet?download=1") # from https://zenodo.org/records/8431323

In [3]:
LIST.head(3)

LIST-ID        EDCS-ID    EDH-ID  \
445463   445464  EDCS-24900077  HD056163   
445464   445465  EDCS-03700724  HD052964   
445465   445466  EDCS-13800065  HD017714   

                                trismegistos_uri pleiades_id  \
445463  https://www.trismegistos.org/text/177366      570485   
445464  https://www.trismegistos.org/text/121715      531064   
445465  https://www.trismegistos.org/text/177100      570049   

                                            transcription  \
445463  Q(uinto) Caecilio C(ai) f(ilio) Metelo / imper...   
445464  Fortissimo et piis/simo Caesari d(omino) n(ost...   
445465  Italicei / quei Aegei negotiantur / P(ublium) ...   

                                              inscription  \
445463  Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...   
445464  Fortissimo et Piis/simo Caesari d(omino) n(ost...   
445465  Italicei / quei Aegei negotiantur / P(ublium) ...   

                                  clean_text_conservative  \
445463  Q Caecilio C f Metelo imperatori Italici quei ...   
445464  Fortissimo et piissimo Caesari d n Gal Val P F...   
445465  Italicei quei Aegei negotiantur P Rutilium P f...   

                         clean_text_interpretive_sentence  \
445463  Quinto Caecilio Cai filio Metelo imperatori It...   
445464  Fortissimo et piissimo Caesari domino nostro G...   
445465  Italicei quei Aegei negotiantur Publium Rutili...   

                             clean_text_interpretive_word  \
445463  Quinto Caecilio Cai filio Metelo imperatori It...   
445464  Fortissimo et piissimo Caesari domino nostro G...   
445465  Italicei quei Aegei negotiantur Publium Rutili...   

                        clean_text_interpretive_word_EDCS  \
445463  Quinto Caecilio Cai filio Metello imperatori I...   
445464  Fortissimo et Piissimo Caesari domino nostro G...   
445465  Italicei quei Aegei negotiantur Publium Rutili...   

                                          diplomatic_text province  \
445463  Q CAECILIO C F METELO / IMPERATORI ITALICI / Q...   Achaia   
445464  FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...   Achaia   
445465  ITALICEI / QVEI AEGEI NEGOTIANTVR / P RVTILIVM...   Achaia   

                                 place        inscr_type  \
445463   Agia Triada / Merbaka / Midea  tituli honorarii   
445464      Agios Athanasios / Photike  tituli honorarii   
445465  Aigio / Egio / Aiyion / Aegeum  tituli honorarii   

                                          status_notation    inscr_process  \
445463  officium/professio, ordo senatorius, tria nomi...             None   
445464       Augusti/Augustae, ordo equester, tria nomina  litterae erasae   
445465  officium/professio, ordo senatorius, tria nomi...             None   

                                                   status  \
445463  officium/professio;  ordo senatorius;  tituli ...   
445464  Augusti/Augustae;  litterae erasae;  ordo eque...   
445465  officium/professio;  ordo senatorius;  tituli ...   

                                             partner_link last_update  \
445463  http://db.edcs.eu/epigr/partner.php?s_language...  2011-11-11   
445464  http://db.edcs.eu/epigr/partner.php?s_language...  2014-09-16   
445465  http://db.edcs.eu/epigr/partner.php?s_language...  2011-03-29   

       letter_size    type_of_inscription         work_status  year_of_find  \
445463        None  honorific inscription            no image           NaN   
445464    3-5.3 cm  honorific inscription  checked with photo           NaN   
445465  3.5-3.7 cm     votive inscription  checked with photo           NaN   

       present_location                                       text_edition  \
445463             None  \n Quinto Caecilio Cai filio Metelo imperatori...   
445464    Fragma Kalama  \n Fortissimo et piissimo Caesari domino nostr...   
445465             None  \n Italicei quei Aegei negotiantur Publium Rut...   

       support_objecttype support_material support_decoration keywords_term  \
445463               N

In [25]:
# list of all columns

print(LIST.columns)

Index(['LIST-ID', 'EDCS-ID', 'EDH-ID', 'trismegistos_uri', 'pleiades_id',
       'transcription', 'inscription', 'clean_text_conservative',
       'clean_text_interpretive_sentence', 'clean_text_interpretive_word',
       'clean_text_interpretive_word_EDCS', 'diplomatic_text', 'province',
       'place', 'inscr_type', 'status_notation', 'inscr_process', 'status',
       'partner_link', 'last_update', 'letter_size', 'type_of_inscription',
       'work_status', 'year_of_find', 'present_location', 'text_edition',
       'support_objecttype', 'support_material', 'support_decoration',
       'keywords_term', 'people', 'type_of_inscription_clean',
       'type_of_inscription_certainty', 'height_cm', 'width_cm', 'depth_cm',
       'material_clean', 'type_of_monument_clean',
       'type_of_monument_certainty', 'province_label_clean',
       'province_label_certainty', 'country_clean', 'country_certainty',
       'findspot_ancient_clean', 'findspot_ancient_certainty',
       'modern_region_cle

### Exploring the dataset

In [ ]:
# number of all inscriptions
len(LIST)

In [8]:
min(LIST.not_before)

-750.0

In [9]:
max(LIST.not_before)

1998.0

In [6]:
min(LIST.not_after)

-671.0

In [7]:
max(LIST.not_after)

2230.0

#  Custom function to extract occupations

In [4]:
occups_declined_dict = json.load(open("../../data/data_generation/occups_declined_dict.json"))

In [5]:
# older functional version
def extract_occup(inscription_text):
    occups_found = []
    if not isinstance(inscription_text, str): # if not valid string
        inscription_text = ""
    for occup in occups_declined_dict.keys():
        for occup_morph in occups_declined_dict[occup]:
            try:
                if occup_morph in inscription_text: # first check it this way, otherwise skip
                    occup_morph_N = len(re.findall("(\W|^)" + occup_morph + "(\W|$)", inscription_text))
                    if occup_morph_N > 0:
                        occups_found.extend([occup] * occup_morph_N)
                        inscription_text = re.sub("(\W|^)(" + occup_morph + ")(\W|$)", r"\1", inscription_text)
            except: pass
    return occups_found

In [6]:
extract_occup("curatores, procuratores et negotiatores curatori navium et curatori")

['curator navium', 'negotiator', 'curator', 'curator']

# Occupational data extraction

In [7]:
# check that our occupations are properly arranged (from the longest...)
list(occups_declined_dict.keys())[:20]

['negotiator artis vestiariae et lintiariae',
 'negotiator artis cretaria et vestiaria',
 'negotiator frumentariae et legumenaria',
 'negotiator salsamentarius et vinarius',
 'negotiator sagarius et pellicarius',
 'negotiator suariae et pecuariae',
 'exactor auri argenti et aeris',
 'negotiator penoris et vinorum',
 'negotiator salsari leguminari',
 'negotiator artis macellariae',
 'negotiator artis purpurariae',
 'negotiator cellarum vinarium',
 'negotiator artis prossariae',
 'negotiator artis vestiariae',
 'negotiator artis ratiariae',
 'inclusor auri et gemmarum',
 'negotiator artis cretaria',
 'negotiator campi pecuarii',
 'negotiator manticularius',
 'negotiator margaritarius']

In [8]:
%%time

# extraction process, takes couple minutes
LIST["occups"] = LIST["clean_text_interpretive_word"].apply(extract_occup)

CPU times: user 3min 8s, sys: 24.6 ms, total: 3min 8s
Wall time: 3min 8s


In [9]:
LIST["occups_N"] = LIST["occups"].apply(len)

In [10]:
# how many times occupation is mentioned
LIST["occups_N"].sum() # LIRE dataset had 5222 instances

10570

In [11]:
# how many inscriptions have at least 1 occupation
len(LIST[LIST["occups_N"]>0]) # LIRE dataset had 4161 inscriptions

8475

In [12]:
# funerary inscriptions
LISToccups = LIST[LIST["occups_N"]>0]

len(LISToccups[LISToccups["type_of_inscription_auto"]=="epitaph"]) 

3116

In [13]:
# honorific inscriptions 
len(LISToccups[LISToccups["type_of_inscription_auto"]=="honorific inscription"]) 

952

In [14]:
# overview of the most common occupations
LIST_occups_list = [el for sublist in LIST["occups"].tolist() for el in sublist]
occupations_counts = pd.DataFrame(nltk.FreqDist(LIST_occups_list).most_common(), columns=["occupation", "count"])
occupations_counts.head(10)

occupation  count
0       curator   1934
1         faber    958
2      aerarius    453
3       medicus    448
4        scriba    421
5   sagittarius    347
6  frumentarius    213
7   centonarius    202
8    negotiator    179
9   argentarius    176

In [15]:
# how many unique occupations there are
len(occupations_counts)

514

In [16]:
#occupations with their counts
occupations_counts

occupation  count
0                     curator   1934
1                       faber    958
2                    aerarius    453
3                     medicus    448
4                      scriba    421
..                        ...    ...
509                funerarius      1
510  negotiator margaritarius      1
511               farmacopola      1
512                  sarcitor      1
513                pectinator      1

[514 rows x 2 columns]

In [17]:
# how many occupations occur only once
len(occupations_counts[occupations_counts["count"]==1])

169

In [18]:
# overview of occupations with their counts
LIST_occups_list = [el for sublist in LIST["occups"].tolist() for el in sublist]
occupations_counts = pd.DataFrame(nltk.FreqDist(LIST_occups_list).most_common(), columns=["occupation", "count"])
occupations_counts.head(10)

occupation  count
0       curator   1934
1         faber    958
2      aerarius    453
3       medicus    448
4        scriba    421
5   sagittarius    347
6  frumentarius    213
7   centonarius    202
8    negotiator    179
9   argentarius    176

## Load the occupation list with all metadata


In [19]:
# load the occupation list with all metadata
occupations_df = pd.read_csv("../../data/data_generation/occupations_list_hisco.csv")
occupations_df.head(5)

Term gen_sg Term2 Vocab_nom_sg             Source  HISCO_majorgroup  \
0    abetarius      i   NaN          NaN  Petrikovits 1981a               8.0   
1   abietarius      i   NaN          NaN  Petrikovits 1981a               8.0   
2     acceptor   oris   NaN     acceptor    Waltzing - Rome               3.0   
3  accomodator   oris   NaN          NaN  Petrikovits 1981a               9.0   
4      aceptor   oris   NaN          NaN  Petrikovits 1981a               3.0   

   HISCO_minorgroup Harris_Category  Subcategory  \
0              81.0        Building  Wood worker   
1              81.0        Building  Wood worker   
2              31.0         Finance          NaN   
3              99.0    Unclassified          NaN   
4              31.0         Finance          NaN   

                   Translation_eng  
0            a joiner, wood worker  
1            a joiner, wood worker  
2  collector, gold quality checker  
3             uncertain, craftsman  
4  collector, gold quality checker

In [20]:
# categorise occupations according to their HISCO group

def get_int(x):
    try: return str(int(x))
    except: return ""
occupations_df["HISCO_majorgroup"] = occupations_df["HISCO_majorgroup"].apply(get_int)
occupations_df["HISCO_minorgroup"] = occupations_df["HISCO_minorgroup"].apply(get_int)

In [21]:
occupations_df.head(5)

Term gen_sg Term2 Vocab_nom_sg             Source HISCO_majorgroup  \
0    abetarius      i   NaN          NaN  Petrikovits 1981a                8   
1   abietarius      i   NaN          NaN  Petrikovits 1981a                8   
2     acceptor   oris   NaN     acceptor    Waltzing - Rome                3   
3  accomodator   oris   NaN          NaN  Petrikovits 1981a                9   
4      aceptor   oris   NaN          NaN  Petrikovits 1981a                3   

  HISCO_minorgroup Harris_Category  Subcategory  \
0               81        Building  Wood worker   
1               81        Building  Wood worker   
2               31         Finance          NaN   
3               99    Unclassified          NaN   
4               31         Finance          NaN   

                   Translation_eng  
0            a joiner, wood worker  
1            a joiner, wood worker  
2  collector, gold quality checker  
3             uncertain, craftsman  
4  collector, gold quality checker

In [22]:
def term1_plus_term2(row):
    term1_2 = row["Term"]
    if isinstance(row["Term2"], str):
        term1_2 += " " + row["Term2"]
    return term1_2

occupations_df["Term"] = occupations_df.apply(lambda row: term1_plus_term2(row), axis=1)

In [23]:
occupation_dict = {}
keys = ["Harris_Category", "Source", "HISCO_majorgroup", "Subcategory", "HISCO_minorgroup", "Translation_eng"]
for n in range(len(occupations_df)):
    occupation_dict[occupations_df.iloc[n]["Term"]] = dict([(key, occupations_df.iloc[n][key]) for key in keys])

In [24]:
occupation_dict

{'abetarius': {'Harris_Category': 'Building',
  'Source': 'Petrikovits 1981a',
  'HISCO_majorgroup': '8',
  'Subcategory': 'Wood worker',
  'HISCO_minorgroup': '81',
  'Translation_eng': 'a joiner, wood worker'},
 'abietarius': {'Harris_Category': 'Building',
  'Source': 'Petrikovits 1981a',
  'HISCO_majorgroup': '8',
  'Subcategory': 'Wood worker',
  'HISCO_minorgroup': '81',
  'Translation_eng': 'a joiner, wood worker'},
 'acceptor': {'Harris_Category': 'Finance',
  'Source': 'Waltzing - Rome',
  'HISCO_majorgroup': '3',
  'Subcategory': nan,
  'HISCO_minorgroup': '31',
  'Translation_eng': 'collector, gold quality checker'},
 'accomodator': {'Harris_Category': 'Unclassified',
  'Source': 'Petrikovits 1981a',
  'HISCO_majorgroup': '9',
  'Subcategory': nan,
  'HISCO_minorgroup': '99',
  'Translation_eng': 'uncertain, craftsman'},
 'aceptor': {'Harris_Category': 'Finance',
  'Source': 'Petrikovits 1981a',
  'HISCO_majorgroup': '3',
  'Subcategory': nan,
  'HISCO_minorgroup': '31',
  '

In [25]:
for key in keys:
    occupations_counts[key] = occupations_counts["occupation"].apply(lambda x: occupation_dict[x][key])
occupations_counts.head(10)

occupation  count         Harris_Category             Source  \
0       curator   1934              Managerial           Waltzing   
1         faber    958            Unclassified    Waltzing - Rome   
2      aerarius    453           Metal-Working    Waltzing - Rome   
3       medicus    448  Miscellaneous Services    Waltzing - Rome   
4        scriba    421               Education    Waltzing - Rome   
5   sagittarius    347           Metal-Working  Petrikovits 1981a   
6  frumentarius    213                  Retail           EDH/EDCS   
7   centonarius    202                Clothing    Waltzing - Rome   
8    negotiator    179                  Retail    Waltzing - Rome   
9   argentarius    176           Metal-Working    Waltzing - Rome   

  HISCO_majorgroup Subcategory HISCO_minorgroup  \
0                2         NaN               21   
1                9         NaN               99   
2                8         NaN               83   
3                0         NaN                6   
4                3         NaN               30   
5                8         NaN               83   
6                4         NaN               43   
7                7         NaN               79   
8                4         NaN               43   
9                8         NaN               88   

                                     Translation_eng  
0  he who takes charge, a manager, overseer, supe...  
1  a worker in wood, stone, metal, etc., a forger...  
2                                       metal worker  
3                                          a surgeon  
4  a public writer, official scribe, professional...  
5                         arrow-makers, arrow-smiths  
6                                   trader with corn  
7             a maker of patchwork, a dealer in rags  
8  one who does business by wholesale, a wholesal...  
9                            jewellery maker, banker

In [26]:
(occupations_counts["count"]==1).sum()

169

In [27]:
# HISCO major groups definition

HISCO_majorgroup_dict = {
    "0" : "Professional, technical and related workers",
    "1" : "Professional, technical and related workers",
    "2" : "Administrative and managerial workers",
    "3" : "Clerical and related workers",
    "4" : "Sales workers",
    "5" : "Service workers",
    "6" : "Agricultural, animal husbandry and forestry workers, fishermen and hunters",
    "7" : "Production and related workers, transport equipment operators and labourers",
    "8" : "Production and related workers, transport equipment operators and labourers",
    "9" : "Production and related workers, transport equipment operators and labourers",
    "" : ""
}

In [28]:
occupations_counts["HISCO_majorgroup_descr"] =  occupations_counts["HISCO_majorgroup"].apply(lambda x: HISCO_majorgroup_dict[x])
occupations_counts.head(5)

occupation  count         Harris_Category           Source HISCO_majorgroup  \
0    curator   1934              Managerial         Waltzing                2   
1      faber    958            Unclassified  Waltzing - Rome                9   
2   aerarius    453           Metal-Working  Waltzing - Rome                8   
3    medicus    448  Miscellaneous Services  Waltzing - Rome                0   
4     scriba    421               Education  Waltzing - Rome                3   

  Subcategory HISCO_minorgroup  \
0         NaN               21   
1         NaN               99   
2         NaN               83   
3         NaN                6   
4         NaN               30   

                                     Translation_eng  \
0  he who takes charge, a manager, overseer, supe...   
1  a worker in wood, stone, metal, etc., a forger...   
2                                       metal worker   
3                                          a surgeon   
4  a public writer, official scribe, professional...   

                              HISCO_majorgroup_descr  
0              Administrative and managerial workers  
1  Production and related workers, transport equi...  
2  Production and related workers, transport equi...  
3        Professional, technical and related workers  
4                       Clerical and related workers

In [7]:
# save a list of occupations with their counts
occupations_counts.to_csv("../../data/data_generation/occupations_counts.csv")

NameError: name 'occupations_counts' is not defined

## Occupations - basic summary

In [50]:
print("LIST - number of occupation occurances: " + str(LIST["occups_N"].sum()))
print("LIST - number of inscriptions with at least one occupation mentioned: " + str(len(LIST[LIST["occups_N"] > 0])))

LIST - number of occupation occurances: 10570
LIST - number of inscriptions with at least one occupation mentioned: 8475


In [51]:
# How many occupations come from the EDH data
LIST[LIST["EDH-ID"].notnull()]["occups_N"].sum() # LIRE had 1272

3139

In [52]:
# How many occupations come from the EDCS data
LIST[LIST["EDH-ID"].isnull()]["occups_N"].sum() # LIRE had 2568

7431

In [53]:
# grouping of occupations by their HISCO major group name
occupations_counts.groupby("HISCO_majorgroup_descr").sum()

occupation  \
HISCO_majorgroup_descr                                                                                  
                                                    faber subaedianusmoritexembaenitariusperfector...   
Administrative and managerial workers               curatorpossessorcircitorhorreariusdissignatorf...   
Agricultural, animal husbandry and forestry wor...  pistorvenatoroleariuspecuariuspiscatorfactorsu...   
Clerical and related workers                        scribaarcariusnummulariusscriptorexceptoraccep...   
Production and related workers, transport equip...  faberaerariussagittariuscentonariusargentarius...   
Professional, technical and related workers         medicusmensorscaenicusgladiatorarchitectusfabe...   
Sales workers                                       frumentariusnegotiatorvestiariusnegotiansmerca...   
Service workers                                     conditorornatrixlecticariuscocusofficinatorred...   

                                                    count  \
HISCO_majorgroup_descr                                      
                                                       10   
Administrative and managerial workers                2153   
Agricultural, animal husbandry and forestry wor...    260   
Clerical and related workers                          705   
Production and related workers, transport equip...   4121   
Professional, technical and related workers          1675   
Sales workers                                         916   
Service workers                                       730   

                                                                                      Harris_Category  \
HISCO_majorgroup_descr                                                                                  
                                                    UnclassifiedUnclassifiedUnclassifiedUnclassifi...   
Administrative and managerial workers               ManagerialManagerialManagerialManagerialManage...   
Agricultural, animal husbandry and forestry wor...  Food-ProductionMiscellaneous ServicesFood-Prod...   
Clerical and related workers                        EducationFinanceFinanceManagerialEducationFina...   
Production and related workers, transport equip...  UnclassifiedMetal-WorkingMetal-WorkingClothing...   
Professional, technical and related workers         Miscellaneous ServicesMiscellaneous ServicesPe...   
Sales workers                                       RetailRetailRetailRetailRetailFood-RetailRetai...   
Service workers                                     Food-ProductionMiscellaneous ServicesFood-Prod...   

                                                                                               Source  \
HISCO_majorgroup_descr                                                                                  
                                                    Ghent Database Roman GuildsPetrikovits 1981aPe...   
Administrative and managerial workers               WaltzingWaltzing - provincesWaltzing - provinc...   
Agricultural, animal husbandry and forestry wor...  Waltzing - RomeWaltzing - RomeWaltzing - RomeW...   
Clerical and related workers                        Waltzing - RomePetrikovits 1981aWaltzing - Rom...   
Production and related workers, transport equip...  Waltzing - RomeWaltzing - RomePetrikovits 1981...   
Professional, technical and related workers         Waltzing - RomeWaltzing - RomeWaltzing - RomeW...   
Sales workers                                       EDH/EDCSWaltzing - RomeWaltzing - RomeWaltzing...   
Service workers                                     Petrikovits 1981aPetrikovits 1981aWaltzing - R...   

                                                                                     HISCO_majorgroup  \
HISCO_majorgroup_descr                                                                                  
                                                                                                        
Administrati

In [6]:
# grouping of occupations by their HISCO major group number
occupations_counts.groupby("HISCO_majorgroup").sum()

NameError: name 'occupations_counts' is not defined

### Exploring individual cases: Faber (worker) and Metal-Working category

In [5]:
# how many times there is a term faber (worker) = generic term for a manual and relativelly unskilled labour
occupations_counts[occupations_counts["occupation"]=="faber"]

NameError: name 'occupations_counts' is not defined

In [56]:
# how many occurences of occupations belonging to the Metal-working category based on Harris 2001
occupations_counts[occupations_counts["Harris_Category"]=="Metal-Working"]["count"].sum()


1221

# Categorisation of urban contexts and industry categories by Harris 2001

In [31]:
# generate a dictionary of occupations by type

def term1_plus_term2(row):
    term1_2 = row["Term"]
    if isinstance(row["Term2"], str):
        term1_2 += " " + row["Term2"]
    return term1_2

occupations_df["Term"] = occupations_df.apply(lambda row: term1_plus_term2(row), axis=1)

occups_cats_dict = dict(zip(occupations_df["Term"], occupations_df["Harris_Category"]))


In [32]:
def cat_for_occup(list_of_occups):
    try:
        return [occups_cats_dict[occup] for occup in list_of_occups]
    except:
        return []
LIST["occups"] = LIST["occups"].apply(list)
LIST["occups_cats_Harris"] = LIST["occups"].apply(cat_for_occup)

# Categorisation of urban contexts and industry categories by HISCO

In [33]:
occupations_df["Term"] = occupations_df["Term"].apply(lambda x: x.replace(" ", "_"))


In [34]:
hisco_cats_dict = {
    0.0 : "Professional, technical and related workers",
    1.0 : "Professional, technical and related workers",
    2.0 : "Administrative and managerial workers",
    3.0 : "Clerical and related workers",
    4.0 : "Sales workers",
    5.0 : "Service workers",
    6.0 : "Agricultural, animal husbandry and forestry workers, fishermen and hunters",
    7.0 : "Production and related workers, transport equipment operators and labourers",
    8.0 : "Production and related workers, transport equipment operators and labourers",
    9.0 : "Production and related workers, transport equipment operators and labourers",
}

In [35]:
hisco_cats_labels = list(set(hisco_cats_dict.values())) + ["Unclassified"]
hisco_cats_labels

['Production and related workers, transport equipment operators and labourers',
 'Service workers',
 'Administrative and managerial workers',
 'Sales workers',
 'Agricultural, animal husbandry and forestry workers, fishermen and hunters',
 'Clerical and related workers',
 'Professional, technical and related workers',
 'Unclassified']

In [36]:
def hisco_literary(hisco_code):
    try:
        return hisco_cats_dict[float(hisco_code)]
    except:
        return "unclassified"
occupations_df["hisco_cats"] = occupations_df["HISCO_majorgroup"].apply(hisco_literary)


In [37]:
occups_cats_dict = dict(zip(occupations_df["Term"], occupations_df["hisco_cats"]))
def cat_for_occup(list_of_occups):
    try:
        return [occups_cats_dict[occup] for occup in list_of_occups]
    except:
        return []
LIST["occups_cats_HISCO"] = LIST["occups"].apply(cat_for_occup)


In [4]:
LIST.head(3)

LIST-ID        EDCS-ID    EDH-ID  \
445463   445464  EDCS-24900077  HD056163   
445464   445465  EDCS-03700724  HD052964   
445465   445466  EDCS-13800065  HD017714   

                                trismegistos_uri pleiades_id  \
445463  https://www.trismegistos.org/text/177366      570485   
445464  https://www.trismegistos.org/text/121715      531064   
445465  https://www.trismegistos.org/text/177100      570049   

                                            transcription  \
445463  Q(uinto) Caecilio C(ai) f(ilio) Metelo / imper...   
445464  Fortissimo et piis/simo Caesari d(omino) n(ost...   
445465  Italicei / quei Aegei negotiantur / P(ublium) ...   

                                              inscription  \
445463  Q(uinto) Caecilio C(ai) f(ilio) Metel(l)o / im...   
445464  Fortissimo et Piis/simo Caesari d(omino) n(ost...   
445465  Italicei / quei Aegei negotiantur / P(ublium) ...   

                                  clean_text_conservative  \
445463  Q Caecilio C f Metelo imperatori Italici quei ...   
445464  Fortissimo et piissimo Caesari d n Gal Val P F...   
445465  Italicei quei Aegei negotiantur P Rutilium P f...   

                         clean_text_interpretive_sentence  \
445463  Quinto Caecilio Cai filio Metelo imperatori It...   
445464  Fortissimo et piissimo Caesari domino nostro G...   
445465  Italicei quei Aegei negotiantur Publium Rutili...   

                             clean_text_interpretive_word  \
445463  Quinto Caecilio Cai filio Metelo imperatori It...   
445464  Fortissimo et piissimo Caesari domino nostro G...   
445465  Italicei quei Aegei negotiantur Publium Rutili...   

                        clean_text_interpretive_word_EDCS  \
445463  Quinto Caecilio Cai filio Metello imperatori I...   
445464  Fortissimo et Piissimo Caesari domino nostro G...   
445465  Italicei quei Aegei negotiantur Publium Rutili...   

                                          diplomatic_text province  \
445463  Q CAECILIO C F METELO / IMPERATORI ITALICI / Q...   Achaia   
445464  FORTISSIMO ET PIIS / SIMO CAESARI D N / GAL VA...   Achaia   
445465  ITALICEI / QVEI AEGEI NEGOTIANTVR / P RVTILIVM...   Achaia   

                                 place        inscr_type  \
445463   Agia Triada / Merbaka / Midea  tituli honorarii   
445464      Agios Athanasios / Photike  tituli honorarii   
445465  Aigio / Egio / Aiyion / Aegeum  tituli honorarii   

                                          status_notation    inscr_process  \
445463  officium/professio, ordo senatorius, tria nomi...             None   
445464       Augusti/Augustae, ordo equester, tria nomina  litterae erasae   
445465  officium/professio, ordo senatorius, tria nomi...             None   

                                                   status  \
445463  officium/professio;  ordo senatorius;  tituli ...   
445464  Augusti/Augustae;  litterae erasae;  ordo eque...   
445465  officium/professio;  ordo senatorius;  tituli ...   

                                             partner_link last_update  \
445463  http://db.edcs.eu/epigr/partner.php?s_language...  2011-11-11   
445464  http://db.edcs.eu/epigr/partner.php?s_language...  2014-09-16   
445465  http://db.edcs.eu/epigr/partner.php?s_language...  2011-03-29   

       letter_size    type_of_inscription         work_status  year_of_find  \
445463        None  honorific inscription            no image           NaN   
445464    3-5.3 cm  honorific inscription  checked with photo           NaN   
445465  3.5-3.7 cm     votive inscription  checked with photo           NaN   

       present_location                                       text_edition  \
445463             None  \n Quinto Caecilio Cai filio Metelo imperatori...   
445464    Fragma Kalama  \n Fortissimo et piissimo Caesari domino nostr...   
445465             None  \n Italicei quei Aegei negotiantur Publium Rut...   

       support_objecttype support_material support_decoration keywords_term  \
445463               N

#   Saving locally

In [39]:
LIST.to_parquet("../../data/large_data/LIST_occups.parquet")

In [3]:
# local version if you have it already locally
LIST = gpd.read_parquet("../../data/large_data/LIST_occups.parquet") 